In [ ]:
from ipywidgets import GridspecLayout
from ipydatagrid import DataGrid, TextRenderer
import ipywidgets as widgets
import ipydatagrid
import pandas as pd
from astropy.table import Table
import eb_predictor
from astropy.time import Time
from spinner_alone import Spinner
from datetime import datetime, UTC

class Page(widgets.Box):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        style = {'description_width': 'initial'}
        #Widget for date picker
        self.obs_date = widgets.DatetimePicker(
            description='Pick an Observing Date and Time',
            value=datetime.now(UTC).astimezone(),
            disabled=False,
            style=style
        )

        # Widget for Days in the Future
        self.days_future = widgets.IntText(
            description="Days in the future",
            value=1,
            disabled = False,
            style=style
        )

        #Widget for Longitude
        self.long=widgets.FloatText(
            description='Longitude',
            disabled=False
        )

        #Widget for Latitude
        self.lat=widgets.FloatText(
            description='Latitude',
            disabled=False
        )

        #Widget for Elevation
        self.elev = widgets.FloatText(
            description='Elevation:',
            disabled=False
        )

        #Widget for Altitude
        self.alti = widgets.FloatText(
            description='Minimum Altitude',
            value=30,
            disabled=False
        )
        self.spinner = Spinner(spinner_file="star_spinner.svg")

        self.table = widgets.VBox([self.spinner], layout={"height": "40em"})

        #Widget for Submit Button
        submit_button = widgets.Button(description='Submit')
        def on_submit(button):

            self.table.children = [self.spinner]
            self.spinner.start()
            self.ebp = eb_predictor.EclipsingBinaryPredictor(
                start_time_str = self.obs_date.value,
                days_in_future=self.days_future.value,
                minimum_altitude = self.alti.value,
                latitude = self.lat.value,
                longitude = self.long.value,
                elevation = self.elev.value,
            )

            self.ebp.init()
            self.ebp.main()

            self.ebp.predictions["Can See"] = "SOMETHING IS WRONG"
            self.ebp.predictions["Can See"][self.ebp.predictions["FirstHalf"]] = "First"
            self.ebp.predictions["Can See"][self.ebp.predictions["SecondHalf"]] = "Second"
            self.ebp.predictions["Can See"][self.ebp.predictions["FullEclipse"]] = "Full"
            self.ebp.predictions["MaxMag"]=[float(star.split()[0]) for star in self.ebp.predictions["MaxMag"]]
            del self.ebp.predictions["ProperMotionRA",
                "ProperMotionDec",
                "VariabilityType",
                "OID",
                "Constellation",
                "EclipseDate",
                "Category",
                "SpectralType",
                "FirstHalf",
                "SecondHalf",
                "FullEclipse",
                "MinMag",
                "Period",
                "Epoch",
                "AUID"
            ]

            df = self.ebp.predictions.to_pandas()
            df["MidtransitTimes"] = [f.round(freq='s') for f in df["MidtransitTimes"]]
            df["EclipseStart"] = [f.round(freq='s') for f in df["EclipseStart"]]
            df["EclipseEnd"] = [f.round(freq='s') for f in df["EclipseEnd"]]
            date_render = TextRenderer(format="%Y-%m-%d %H:%M:%S", format_type="time")
            renderer = {"MidtransitTimes": date_render, "EclipseStart": date_render, "EclipseEnd": date_render}
            self.dg2 = DataGrid(df, renderers=renderer)
            self.table.children = [self.dg2]
            self.dg2.auto_fit_columns = True
        submit_button.on_click(on_submit)
        self.submit = submit_button

        # Creates Widget Grid
        self.grid = GridspecLayout(5, 4, height='400px')
        self.grid[0, 1] = self.obs_date
        self.grid[1,1] = self.days_future
        self.grid[2, 0] = self.elev
        self.grid[0, 0] = self.long
        self.grid[1, 0] = self.lat
        self.grid[2, 1] = self.alti
        self.grid[1, 2] = self.submit
        self.grid[4, :3] = self.table

        self.children = [self.grid]


# Goals:
# Single object functionality?

In [ ]:
eb = Page()
eb